### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2018 ###

# Project title goes here #

#### Zijie (Jay) Wang (zwang688@wisc.edu)

*****

### Table of Contents

## 1. Introduction ##

The first few sentences should give a quick overview of the entire project. Then, elaborate with a description of the problem that will be solved, a brief history (with [citations](https://en.wikipedia.org/wiki/Citation)) of how the problem came about, why it's important/interesting, and any other interesting facts you'd like to talk about. You should address and explain where the problem data is coming from (research? the internet? synthetically generated?) Also give an outline of the rest of the report.*

This section should be 300-600 words long, and **should be accessible to a general audience** (don't assume your reader has taken the class!). Feel free to include images if you think it'll be helpful.


## 2. Mathematical Model ##

A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**.


For the convenience of notation, we define $\mathcal{I}$ as the set containing labels of all student enrolled in CS506, and $\mathcal{J}$ as the set of all projects which are selected for billing. 

### 2.1. Modeling Assumptions

- **Integer Programming**

    In this problem, we are assigning students into groups, so we expect integer solutions.

### 2.2. Decision Variables

The decision variable is a matrix $A \in M_{\Vert \mathcal{I} \Vert \times \Vert \mathcal{J} \Vert}\left(\text{Bin}\right)$.

$$
A_{i,j}=\begin{cases}
           1 &\text{If student $i$ is assigned to project $j$}\\
           0 &\text{Otherwise}
        \end{cases}
$$

### 2.3. Constraints

- **Group Size Constraint**

    Each group can only have $l$ to $u$ ($l, u \in \mathbb{N}, u > l$) members, where $i, j$ are decided by the instructor. However, some projects will be discarded in this assignment, so each project should have variable size either $0$ or $l$ to $u$.
    
    For example, there are $90$ students enrolled in CS506 in 2018 Spring, and two TA's have chosen $29$ candidate project proposals. Each project requires at least $5$ students (at most $18$ final projects), so at least $11$ project will be discarded. 
    
    In other words, $$\forall j \in \mathcal{J}, \quad \sum_{i=1}^{m}A_{i,j} = 0 \quad \text{or} \quad  l \leq \sum_{i=1}^{m}A_{i,j} \leq u$$

<a id='constraints'></a>
- **Student Choice Constraint**

    Each student will be assigned to exactly one group,
    $$\forall i \in \mathcal{I}, \quad \sum_{j=1}^{n}A_{i,j} = 1$$
    
- **Proposal Writer Constraint**

    The instructor of CS506 made a rule that the person who submits the proposal must be working on his/her submitted project, if it won the bidding. 
    
    We define a discrete injective function $F_w: \mathcal{J} \mapsto \mathcal{I}$ to map each project to its writer. Then, we can model this constraint as a logical constraint,
    $$\forall j \in \mathcal{J}, \quad \sum_{i=1}^{m}A_{i,j} \ne 0 \longrightarrow A_{F_w\left(j\right), j} = 1$$

### 2.4. Objective Function

At the beginning of the billing phase, all students in CS506 are asked to complete a background survey. We can use the information to define the following five discrete surjective functions.

1. $F_1: \mathcal{I} \mapsto \{1, 2, 3, 4, 5\}$, mapping each student to the scale of the most complicated project he/she has done.
2. $F_2: \mathcal{I} \mapsto \{1, 2, 3, 4, 5\}$, mapping each student to the level of his/her lifetime programming experience.
3. $F_3: \mathcal{I} \mapsto \mathcal{J}^8$, mapping each student to an ordered 8-tuple of project numbers that he/she is interested.
4. $F_4: \mathcal{I} \mapsto \mathcal{I}^3$, mapping each student to an ordered 3-tuple of students who he/she wants to work with.
5. $F_5: \mathcal{I} \mapsto \mathcal{I}^2$, mapping each student to an ordered 2-tuple of students who he/she does not want to work with.

We can also introduce a weight value for each of the function above, to indicate the importance level for each measure. In this model, we assume all weights are fixed and given by the instructor.

1. $\beta_1 \in \mathbb{R}:$ scalar weight of $F_1$.
2. $\beta_2 \in \mathbb{R}:$ scalar weight of $F_2$.
3. $\beta_3 \in \mathbb{R}^8:$ an ordered 8-tuple weight of $F_3$.
4. $\beta_4 \in \mathbb{R}^3:$ an ordered 3-tuple weight of $F_4$.
5. $\beta_5 \in \mathbb{R}^2:$ an ordered 2-tuple weight of $F_5$.

All weights $\beta_1, \dots, \beta_5$ can be positive or negative.


Taking advantage of the fact that all decision variables are binary and $\mathcal{I}$ being determinative and finite, we can vectorize these mappings to make the transformations of $A$ become easy linear algebras. To make the notations simple, we use the same notation $\left(F_1,\dots, F_5\right)$ for the vectorized functions. The specific vectorization is discussed in the section below.

#### 2.4.1 Baseline model

For the baseline model, we consider the fairness score $\left(O\right)$ as a linear combination of diversity score $\left(D\right)$ and student happiness score $\left(H\right)$. We assume only $F_1$ and $F_2$ contribute to the programing experience diversity of each group, and only $F_3, F_4, F_5$ contribute to the student happiness of their group assignments.

- **Experience Diversity**

    We can first vectorize functions $F_1, F_2$ into two column vectors with length $i$, then to get the mean experience score $\left(E\right)$ of all $j$ groups with regarding to the weights $\beta_1, \beta_2$,

    $$E = \text{diag}\left( A^T \times \vec{1} \right)^{-1} \left(A^T\left(\beta_1 F_1\right) + A^T\left(\beta_2 F_2\right)\right)$$

    It's worth mentioning that $\vec{1}$ is a length $i$ column vector, so $A^T \times \vec{1}$ becomes a length $j$ column vector of $j$ group sizes. Then, $\text{diag}\left( A^T \times \vec{1} \right)^{-1}$ is a diagonal matrix whose diagonal is exactly the inverse of the group sizes. Finally $E$ is a length $j$ vector of the experience average of all groups.
    
    The programming experience diversity $\left(D\right)$ then can be modeled as the variance of those averages. Denote $\overline{E}$ as the population mean of the averages of all groups' experience level,

    $$\overline{E} = \frac{\sum_{k=1}^j E_k}{j}$$

    Then, we can write the variance as,
    $$
    \begin{align*}
    D &= \text{Var}\left(E \right)\\
        &= \frac{1}{j-1}\sum_{k=1}^j\left(E_k - \overline{E}\right)^2
    \end{align*}
    $$

- **Student Happiness**

    Student happiness is defined as the weighted matches of $F_3\left(\mathcal{I}\right), F_4\left(\mathcal{I}\right), F_5\left(\mathcal{I}\right)$ and student's real assignments. 

    For example, student $\mathcal{I}_p$ has an ordered list of projects $F_3\left(\mathcal{I}_p\right)=\begin{bmatrix}a_1\\a_2\\a_3\\q\\a_5\\a_6\\a_7\\a_8\end{bmatrix}$ he wants to work on, and sorted lists $F_4\left(\mathcal{I}_p\right) = \begin{bmatrix}b_1\\b_2\\b_3\end{bmatrix}$ and  $F_5\left(\mathcal{I}_p\right) = \begin{bmatrix}p_4\\c_2\end{bmatrix}$ of students he would like/ not like to be grouped with. He is assigned to group $q$ with other students $p_1, \dots, p_5$. Suppose he did rank $q$ as his fourth favorite project, $F_1^{\left(4\right)}\left(\mathcal{I}_p\right) = q$, then there is a happiness reward $\beta_3^{\left(4\right)}$ for him. All the students he wants to work with are not assigned to his group, and the student $p_3$, who he doesn't like, is in his group. Therefore, there is no happiness bonus but penalty $\beta_5^{\left(1\right)}$ from $F_4, F_5$.
    
    To make the algebraic expression simple, we can vectorize $F_3$ and $\theta_3$ (note they are all given) into a matrix $F_3 \in M_{\Vert \mathcal{I} \Vert \times \Vert \mathcal{I} \Vert}\left(\beta_3\right)$.

    $$
    F_{3_{k,l}}=\begin{cases}
               \beta_3^{\left(p\right)} &\text{If student $k$ has listed student $l$ in his/her preference list with rank $p$}\\
               0  &\text{If student $l$ is not on the preference list of student $k$}
            \end{cases}\\
    $$

    By this [lemma 1](#lemma1) below, the happiness score from $F_3$ can be written as below (note $\circ$ is Hadamard multiplication, a.k.a the element-wise matrix multiplication),
    
    $$H_{F_3} = \sum_{l\in\mathcal{I}, k\in\mathcal{I}}\left(AA^T \circ F_3\right)_{l,k}$$
    
    We then vectorize $F_4$ and $\theta_4$ into a matrix $F_4 \in M_{\Vert \mathcal{I} \Vert \times \Vert \mathcal{J} \Vert}\left(\beta_4\right)$.
    
    $$
    F_{4_{i,j}}=\begin{cases}
               \beta_4^{\left(i\right)} &\text{If student $i$ has listed project $j$ in his/her preference list with rank $p$}\\
               0  &\text{If project $j$ is not on the preference list of student $i$}
            \end{cases}\\
    $$
    
    The happiness reward from $F_4$ simply becomes,
    
    $$H_{F_4} = \sum_{i\in\mathcal{I}, j\in\mathcal{J}}\left(A \circ F_4\right)_{i, j}$$
    
    Using the same vectorization as $F_4$, the happiness penalty from $F_5$ is,
    
    $$H_{F_5} = \sum_{i\in\mathcal{I}, j\in\mathcal{J}}\left(A \circ F_5\right)_{i, j}$$
    
    Finally, the total happiness score is,
    
    $$H = H_{F_3} + H_{F_4} + H_{F_5} =  \sum_{l\in\mathcal{I}, k\in\mathcal{I}}\left(AA^T \circ F_3\right)_{l,k} + \sum_{i\in\mathcal{I}, j\in\mathcal{J}}\left(A \circ F_4\right)_{i, j} + \sum_{i\in\mathcal{I}, j\in\mathcal{J}}\left(A \circ F_5\right)_{i, j}$$

- **Combination**

    Intuitively we think there is rather a trade-off relationship between course diversity and student happiness. We can give them two trade-off parameters $\delta_1, \delta_2 \in \mathbb{R}$.
    
    Then the objective function $\left(O\right)$ is,
    
    $$
    \begin{align*}
    O &= \delta_1 D + \delta_2 H \\
        &= \delta_1 \frac{1}{j-1}\sum_{k=1}^j\left(E_k - \overline{E}\right)^2 + \delta_2 \left(\sum_{l\in\mathcal{I}, k\in\mathcal{I}}\left(AA^T \circ F_3\right)_{l,k} + \sum_{i\in\mathcal{I}, j\in\mathcal{J}}\left(A \circ F_4\right)_{i, j} + \sum_{i\in\mathcal{I}, j\in\mathcal{J}}\left(A \circ F_5\right)_{i, j}\right)
    \end{align*}
    $$

<a id='lemma1'></a>
**Lemma 1:** The matrix $AA^T$  maps students to their group members.

**Proof:** The $k$th row of $AA^T$ is $\big[\langle A_{\left(k,*\right)},  A_{\left(1,*\right)}\rangle, \langle A_{\left(k,*\right)},  A_{\left(2,*\right)}\rangle, \dots, \langle A_{\left(k,*\right)},  A_{\left(i,*\right)}\rangle\big]$.

The dot product $\langle A_{\left(k,*\right)},  A_{\left(i,*\right)}\rangle = \sum_{p=1}^i A_{k,p}A_{i,p}$.

1. From the [Student Choice Constraint](#constraints), we know there is only one $1$ in each row of $A$. Also, $1\times 0 = 0, 1 \times 1 = 1$. Therefore, each dot product can only be $1$ or $0$. In other words, $AA^T$ is binary.

2. We know $\sum_{p=1}^i A_{k,p}A_{i,p} = 1$ if and only there is one $p$ such that $A_{k,p} = A_{i,p} = 1$. It implies the dot product becomes $1$ if and only there is a group $p$ such that both student $k$ and student $i$ are assigned to.

Therefore, 
    $$
\left(AA^T\right)_{\left(k,l\right)}=\begin{cases}
           1 &\text{If student $k$ and $l$ are in the same group}\\
           0  &\text{Otherwise}
        \end{cases}\\
    $$

**Example:**

 Suppose $A = \begin{bmatrix}1&0\\0&1\\0&1\\1&0 \end{bmatrix},\quad AA^T = \begin{bmatrix}1&0\\0&1\\0&1\\1&0 \end{bmatrix}\begin{bmatrix}1&0&0&1\\0&1&1&0 \end{bmatrix} = \begin{bmatrix}1&0&0&1\\0&1&1&0\\0&1&1&0\\ 1&0&0&1\end{bmatrix}$

### 2.5. Standard Forms

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

### 3.1. Model with only 

In [1]:
using JuMP

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.